<h1 style="font-size: 3em;">Algoritmo de agrupamiento CK-Means</h1>  

# 0) Marco teórico

### <br>0.1) Introducción: _Clustering_ y problema de particionamiento

El _clustering_ es una técnica de aprendizaje no supervisado cuyo objetivo es agrupar datos en subconjuntos homogéneos, de forma que los elementos que queden ubicados dentro de un mismo grupo sean más similares entre sí que con los elementos de otros grupos.  

En particular, los algoritmos de _clustering_ de tipo particionamiento dividen un conjunto de **N** datos en **k** grupos que son disjuntos entre sí.  

Para estudiar la similitud entre los datos, se pueden utilizar distintos criterios, siendo uno de los más conocidos y empleados el que se basa en la distancia euclidiana. &nbsp;En los métodos de particionamiento, esta distancia se utiliza para definir una función objetivo global, conocida como SSE _(Sum of Squared Errors)_, la cual cuantifica la dispersión _intra-cluster_, y que se define matemáticamente como:

##### $$SSE = \sum_{j=1}^k \sum_{x_{i} \in C_{j}} (x_{i} - \mu_{j})^2$$   
en donde $C_{j}$ representa el conjunto de elementos del cluster $j$ junto a su centroide $\mu_{j}$.  




### 0.2) Algoritmo de _clustering_ K-Means (clásico)

El algoritmo **K-means** es uno de los métodos de _clustering_ más conocidos y utilizados.  Su funcionamiento se basa en un proceso que consta de dos partes: la inicialización de los centroides de cada _cluster_ (suelen generarse aleatoriamente), y el posterior subproceso iterativo que alterna dos pasos fundamentales:  

1. Asignación: Cada dato (observación) se asigna al _cluster_ cuyo centroide esté más cercano según la distancia euclidiana.  
2. Actualización: Los centroides se recalculan como la media de los puntos asignados a cada _cluster_.  

Este procedimiento se repite hasta que las asignaciones dejen de cambiar o se alcance un criterio de convergencia definido.  Si bien **K-means** es computacionalmente eficiente y no muy difícil de implementar, presenta una importante limitación, que es que la solución depende bastante de la inicialización de los centroides que se hace al inicio, lo cual puede hacer que no siempre se obtenga un particionamiento óptimo de los datos.  

Esta limitación se vuelve especialmente importante cuando los datos son de carácter unidimensional, en donde el problema de particionamiento sí admite soluciones óptimas y exactas.

#### **Pseudocódigo de K-Means (ejemplo en 1 dimensión)**
Aclaración: Es solo un pseudocódigo.
```javascript
/**
 * @param list Lista con los datos.
 * @param k Nro de clusters.
**/
function k-means(data as list, k as integer):

    // Contador de interaciones
    iter as static integer = 0
    
    // Creación de centroides y grupos
    centroides = listObject(size=k, init=zeros())
    grupos = listObject(size=k, init=classObject(atributes=[.centroide=0, .datos=newEmptyListObject()]))  

    // Inicialización aleatoria del primer centroide
    centroides[0] = random(range(list.minvalue(), list.maxvalue()))
    grupos[0].setCentroide(centroides[0])

    // Inicialización aleatoria de los otros centroides
    for i in range(1, k, includeLower=true, includeUpper=false):
        do:
            centroides[i] = random(range(list.minvalue(), list.maxvalue()))
        while:
            centroides[i] in centroides
        grupo[i].setCentroide(centroides[i])

    do:
        if iter != 0:
            old_sse = sum(sum((x-x.grupo.centroide())^2) for x in grupo, for grupo in grupos)

        else:
            old_sse = 0  // Por defecto

        // Limpieza de los datos de los grupos
        for grupo in grupos:
            grupo.datos.clear()
            
        // Asignación a distintos clusters
        for x in data:
            minDistance = +infinity
            bestGroup = None
            for grupo in grupos:
                distance = abs(x - grupo.getCentroide())
                if distance < min_distance:
                    minDistance = distance
                    bestGroup = grupo
            bestGroup.addDato(x)
    
        // Actualización de centroides
        for grupo in grupos:
            grupo.setCentroide(grupo.datos.mean())
        
        iter++
    
    while:
        abs(sum(sum((x-x.grupo.centroide())^2) for x in grupo, for grupo in grupos) - old_sse)/old_sse > 0.05

    return grupos

### 0.3) Algoritmo de _clustering_ CK-Means en una dimensión

CK-Means es un algoritmo de _clustering_ que es una variante optimizada del clásico algoritmo de agrupamiento de K-Means cuando los datos son unidimensionales.  

En este algoritmo, primero se ordenan los datos, y luego mediante programación dinámica, se halla la segmentación óptima de los datos, tratándolos como si los mismos estuviesen ubicados en una recta numérica.  

A diferencia de K-means clásico, CK-means 1D:  

* No depende de centroides iniciales. 
* No utiliza un proceso iterativo de asignación-actualización. 
* No incorpora aleatoriedad. 
* Garantiza la solución globalmente óptima respecto a la función objetivo SSE.  

Formalmente, el problema se expresa como:  

##### $$\min_{C_{1},...,C_{k}} \sum_{j=1}^k SSE(C_{j})$$

sujeto a que cada $C_{j}$ sea un intervalo contiguo de los datos ordenados.

Una vez determinada la partición óptima, los centroides se calculan a posteriori como la media de cada intervalo.

#### **Pseudocódigo de CK-Means**
``` javascript
/**
 * CK-means (1D) por Programación Dinámica
 * @param data Lista con los datos (1D).
 * @param k Nro de clusters.
**/
function ckmeans(data as list, k as integer):

    // Precondición: CK-means en 1D requiere ordenar los datos
    data_sorted = sort(data, ascending=true)

    n = data_sorted.size()

    // Preparación de las sumas acumuladas para calcular SSE(i,j)
    S1 = listObject(size=n+1, init=zeros())
    S2 = listObject(size=n+1, init=zeros())

    S1[0] = 0
    S2[0] = 0

    // Se calculan las sumas acumuladas
    for t in range(1, n+1, includeLower=true, includeUpper=false):
        x = data_sorted[t-1]
        S1[t] = S1[t-1] + x
        S2[t] = S2[t-1] + x^2


    // SSE de un segmento contiguo.  Fórmula: SSE(i,j) = sum(x^2) - (sum(x)^2)/m, con m = (j-i+1)
    function sse_segment(i as integer, j as integer):
        sum_x  = S1[j] - S1[i-1]
        sum_x2 = S2[j] - S2[i-1]
        m = (j - i + 1)
        return sum_x2 - (sum_x)^2/m

    // 2) Tablas DP
    // DP[r][j] = SSE mínimo al particionar los primeros j datos en r clusters
    DP   = listObject(size=k+1, init=listObject(size=n+1, init=+infinity))
    PREV = listObject(size=k+1, init=listObject(size=n+1, init=-1))  // para reconstrucción

    // Caso base: 1 cluster (r=1)
    DP[1][0] = 0  // 0 datos => costo 0 (convención útil)
    for j in range(1, n+1, includeLower=true, includeUpper=false):
        DP[1][j] = sse_segment(1, j)
        PREV[1][j] = 0  // corte anterior en 0 (todo el segmento es 1..j)

    // 3) Llenado de DP para r = 2..k
    for r in range(2, k+1, includeLower=true, includeUpper=false):

        DP[r][0] = 0  // 0 datos => costo 0 (convención)
        
        // Importante: para formar r clusters con j datos, debe cumplirse j >= r
        for j in range(1, n+1, includeLower=true, includeUpper=false):

            if j < r:
                DP[r][j] = +infinity
                PREV[r][j] = -1
            else:
                // DP[r][j] = min_{t = r-1..j-1} DP[r-1][t] + SSE(t+1, j)
                bestCost = +infinity
                bestCut = -1

                for t in range(r-1, j, includeLower=true, includeUpper=false):
                    cost = DP[r-1][t] + sse_segment(t+1, j)
                    if cost < bestCost:
                        bestCost = cost
                        bestCut = t

                DP[r][j] = bestCost
                PREV[r][j] = bestCut

    // 4) Reconstrucción de clusters desde PREV
    grupos = listObject(size=k, init=classObject(atributes=[.datos=newEmptyListObject(), .centroide=0]))

    r = k
    j = n

    while r >= 1:
        t = PREV[r][j]  // corte anterior
        // el cluster r contiene (t+1 .. j) en data_sorted
        grupoDatos = newEmptyListObject()

        for idx in range(t+1, j+1, includeLower=true, includeUpper=false):
            grupoDatos.add(data_sorted[idx-1])

        grupos[r-1].datos = grupoDatos
        grupos[r-1].centroide = mean(grupoDatos)  // en 1D el centroide es la media

        j = t
        r = r - 1

    return grupos
```


### 0.4) Ejemplo teórico (hecho "a mano")

Supongamos que tenemos los siguientes datos no agrupados:  

0, 4, 2, 5, 11, 3

Y queremos dividir este conjunto pequeño de 6 datos, en 3 _clusters_.

##### **Paso 1:** Ordenar los datos

Ahora los datos quedan ordenados de la siguiente forma:

0, 2, 3, 4, 5, 11

##### **Paso 2:** Hacer una tabla con los datos y sus sumas acumuladas

| $i$ | $x_{i}$ | $S_{i}$ | $Q_{i}$
|:---:|:-------:|:-------:|:-------:
|  1  |    0    |    0    |    0
|  2  |    2    |    2    |    4
|  3  |    3    |    5    |   13
|  4  |    4    |    9    |   29
|  5  |    5    |   14    |   54
|  6  |   11    |   25    |  175




## 1. Importación de librerías necesarias

In [1]:
import pandas as pd
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from ckmeans import ckmeans


In [2]:
data = pd.read_csv("data.csv")
data


,Persona,Presión sistólica,Presión diastólica
0,Ana,139,84
1,Pedro,128,92
2,Kevin,117,78
3,Julieta,126,85
4,Héctor,149,97
5,Tamara,142,88
6,Norberto,174,105
7,Olivia,107,59
8,Gerardo,126,70
9,Camila,130,82


In [3]:
data.shape

(15, 3)

In [4]:
data.dtypes

Persona               object
Presión sistólica      int64
Presión diastólica     int64
dtype: object

In [5]:
summary = data.describe().T
summary

,count,mean,std,min,25%,50%,75%,max
Presión sistólica,15.0,136.000000,18.106826,107.0,125.5,130.0,145.5,174.0
Presión diastólica,15.0,83.266667,12.651746,59.0,77.5,84.0,91.0,105.0


In [17]:
presiones = data[["Presion_diastolica", "Presion_sistolica"]].values
scaler = StandardScaler()
presiones_std = scaler.fit_transform(presiones)
print("Medias:", scaler.mean_)
print("Desv.est:", scaler.scale_)
presiones_std

Medias: [136.          83.26666667]
Desv.est: [17.49285568 12.22274746]


array([[ 0.17149859,  0.05999742],
       [-0.45732956,  0.71451475],
       [-1.08615771, -0.43089057],
       [-0.57166195,  0.14181209],
       [ 0.74316054,  1.12358808],
       [ 0.34299717,  0.38725609],
       [ 2.17231541,  1.77810541],
       [-1.65781966, -1.98536923],
       [-0.57166195, -1.0854079 ],
       [-0.34299717, -0.10363191],
       [ 1.48632107,  0.55088542],
       [ 1.08615771, -0.26726124],
       [-0.91465912, -1.5762959 ],
       [ 0.22866478,  1.20540274],
       [-0.62882815, -0.51270524]])

In [19]:
pca = PCA(n_components=1, random_state=42)
pc1 = pca.fit_transform(presiones_std).ravel()  # vector 1D
pc1

array([ 0.1636924 ,  0.18185739, -1.07271513, -0.30394975,  1.3199906 ,
        0.51636703,  2.79336935, -2.57612356, -1.17172533, -0.31581445,
        1.44052252,  0.57904724, -1.76137118,  1.01403887, -0.807186  ])

In [20]:
ckmeans(pc1, 3)

[array([-2.57612356, -1.76137118, -1.17172533, -1.07271513, -0.807186  ]),
 array([-0.31581445, -0.30394975,  0.1636924 ,  0.18185739,  0.51636703,
         0.57904724]),
 array([1.01403887, 1.3199906 , 1.44052252, 2.79336935])]